1) naive bayes simple sentiment analysis with NLTK

https://www.datacamp.com/community/tutorials/simplifying-sentiment-analysis-python

In [ ]:
import nltk
from nltk.corpus import movie_reviews
import random

In [ ]:
#dataset load
nltk.download('movie_reviews')
documents = [(list(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]

random.shuffle(documents) #

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [ ]:
#feature extraction/get list of most frequent words in the corpus
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000]#top 2000 words

def document_features(document):
  document_words = set(document)
  features = {}
  for word in word_features:
    features['contains({})'.format(word)] = (word in document_words)
  return(features)
#use function document features to... return which of the top 2000 words are in doc? 

In [ ]:
word_features[0:10]

['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party']

In [ ]:
# Train Naive Bayes classifier
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
#test classifier: sorting features? 
print(nltk.classify.accuracy(classifier, test_set))

0.75


In [ ]:
classifier.show_most_informative_features(5)

Most Informative Features
     contains(atrocious) = True              neg : pos    =     11.0 : 1.0
           contains(ugh) = True              neg : pos    =      9.0 : 1.0
 contains(unimaginative) = True              neg : pos    =      8.3 : 1.0
        contains(shoddy) = True              neg : pos    =      7.0 : 1.0
    contains(schumacher) = True              neg : pos    =      6.6 : 1.0


shows that if the review contains 'atrocious' it is 11 times more likely to be negative rather than positive

can then... assign scores to see which future reviews contain those high impact words? arbitrary points/values of positivity/negativity??? 

so in theory you would use this approach to train a classifier on sample dataset of tweets or something, then apply the features to evaluate future documents???? 

still not sure how to take the features from this classifier into another thing... 

NLTK basic naive bays classification 2, from textbook
https://www.nltk.org/book/ch06.html

In [ ]:
#from nltk.corpus import names
nltk.download('names')
labeled_names = ([(name, 'male') for name in names.words('male.txt')] + 
[(name, 'female') for name in names.words('female.txt')])
random.shuffle(labeled_names)

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


In [ ]:
labeled_names[0:5] 
#creates a list of classified names as m/f 

[('Thorpe', 'male'),
 ('Roosevelt', 'male'),
 ('Anne-Corinne', 'female'),
 ('Marcello', 'male'),
 ('Robbert', 'male')]

In [ ]:
def gender_features(word):
  return{'last_letter': word[-1]}
#use function to get specific features out of inputs, room to customize here


In [ ]:
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier2 = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
classifier2.classify(gender_features('Joe')) 
#last letter recognition isn't that great by itself

'female'

In [ ]:
print(nltk.classify.accuracy(classifier2, test_set))

0.758


Let's change the number of features that get input and test again

In [ ]:
#vowel counter
def vcount(input):
  vowel_counts={}
  for vowel in 'aeiou':
    count = input.count(vowel)
    vowel_counts[vowel] = count
  return(sum(vowel_counts.values()))


In [ ]:
def gender_features(word):
  return{'last_letter': word[-1], 
         'length':len(word), 
         'first_letter': word[0], 
         'vowel_count':vcount(word)}

featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier2v2 = nltk.NaiveBayesClassifier.train(train_set)

print(nltk.classify.accuracy(classifier2, test_set))

0.772


In [ ]:
classifier2v2.classify(gender_features('joe')) #better

'male'

In [ ]:
#classifier evaluation 
test_set = [(gender_features(n), gender) for (n, gender) in labeled_names]
classifier3 = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier3, test_set))

0.7706445115810675


In [ ]:
errors = []
for (name, tag) in labeled_names:
  guess = classifier3.classify(gender_features(name))
  if guess !=tag:
    errors.append((tag, guess, name))
for (tag, guess, name) in sorted(errors):
  print('correct = {:<8} guess = {:<8} name={:<30}'.format(tag, guess, name))

correct = female   guess = male     name=Abbey                         
correct = female   guess = male     name=Adah                          
correct = female   guess = male     name=Addis                         
correct = female   guess = male     name=Adel                          
correct = female   guess = male     name=Adelind                       
correct = female   guess = male     name=Adey                          
correct = female   guess = male     name=Adriaens                      
correct = female   guess = male     name=Adrian                        
correct = female   guess = male     name=Adrien                        
correct = female   guess = male     name=Ag                            
correct = female   guess = male     name=Agnes                         
correct = female   guess = male     name=Aidan                         
correct = female   guess = male     name=Aigneis                       
correct = female   guess = male     name=Ailyn                  

Use this output to manually tune classifier parameters. Can look at stuff like ann being classified as male to add more parameters like first 2 or last 2 letters or specify rults like 'lyn' being more femail than male

twitter analysis - https://towardsdatascience.com/creating-the-twitter-sentiment-analysis-program-in-python-with-naive-bayes-classification-672e5589a7ed
???


viz class idea? 

- most common word analysis on political figure speeches

- sentiment analysis on those most common words? associations of those words/analysis, like reading level, emotional assignment? fear/hope/hate/peace, ect?? is that even a thing? 

- grab speeches from senators/supreme court justices, ect, and test for significant partisan divide between affiliations in word useage and sentiment afilliation

- present as two party count of ... something???? ideally end up with a frame of political figures and speech statistics that can be aggregated ? 